In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import deepdish as dd
import csv
import random
import os
import sys
from sklearn import mixture
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import silhouette_score
from sklearn.metrics import mean_squared_error

from collections import deque
import torch
import torch.nn as nn
import torch.nn.functional as F 

import torch.autograd
from torch.autograd import Variable
import torch.optim as optim
import math
from collections import namedtuple, deque
from scipy import stats

import gurobipy  as grb
from gurobipy import *
GRB = grb.GRB

import statistics
from cvxopt import matrix, solvers
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from scipy import io

In [2]:
files = os.listdir('D:\python\聚类Exploration\Baseline估计Integrate Way\CleanData')
#print(files)
print(len(files))

Dicblock0 = {}
for i in range(len(files)):
    Dicblock0[files[i]] = np.load("D:/python/聚类Exploration/Baseline估计Integrate Way/CleanData/" + files[i])

print('The number of users:', len(files))


with open('D:/python/Online Forecast/Elasticity Modelling/Acorn.csv') as csv_file:
    reader = csv.reader(csv_file)
    Acorn_info = dict(reader)
    
    
with open('D:/python/Online Forecast/Elasticity Modelling/ToU.csv') as csv_file:
    reader = csv.reader(csv_file)
    ToU_info = dict(reader)

df_tarrif = pd.read_csv("D:/python/Online Forecast/Elasticity Modelling/Tariffs.csv", encoding="utf-8")

1132
The number of users: 1132


In [3]:
# 判断 Acorn_info, ToUInfo unique类别的个数 
def unique(Dic):
    lis = []
    for key,value in Dic.items():
        lis.append(value)
    #print(lis)
    df = pd.DataFrame(lis)
    return df[0].unique()

def CustomerDivisonTariff(ToU_info, ToUUnique,Dicblock0):
    
    ## 初始化
    TariffDic = {}
    TariffDicStdR = {}
    TariffDicToUR = {}
    
    TariffDicStdMAX = {}
    TariffDicToUMAX= {}
    
        
    for x in ToUUnique:
        TariffDic[x] = []
     
        
    ## 划分三个集合    
    for key,value in ToU_info.items():
        if value == 'Std':
            TariffDic['Std'].append(Dicblock0[key+'.npy'])
        else:
            TariffDic['ToU'].append(Dicblock0[key+'.npy'])
            
    print('Number of ToU users:',len(TariffDic['ToU']))
    print('Number of Std users:',len(TariffDic['Std']))
    
      
    
    s = 0
    for user in range(len(TariffDic['ToU'])):
        s += TariffDic['ToU'][user]
    
    sStd = 0
    for user in range(len(TariffDic['Std'])):
        sStd += TariffDic['Std'][user]
    
    
    
    
    return s+20



In [4]:
def load_normalization(load,hmin,hmax):
    
    load = load.reshape(-1)
    Xmin = np.min(load)
    Xmax = np.max(load)
    
    XR = (load-Xmin)/(Xmax-Xmin)
    
    print(np.min(XR*(hmax-hmin)+hmin),np.max(XR*(hmax-hmin)+hmin))
    
    return (XR*(hmax-hmin)+hmin).reshape(365,-1)

In [5]:
ToUUnique = unique(ToU_info)
TariffDicToUEst = CustomerDivisonTariff(ToU_info, ToUUnique,Dicblock0)
TariffDicToUEst = load_normalization(TariffDicToUEst[:,np.arange(0,48,2)],hmin = 55,hmax = 90)

Number of ToU users: 251
Number of Std users: 881
55.0 90.0


In [6]:
wind2012 = np.load('D:\python\Integrated Forecasting and Optimizing\Imbalance Example/clean data/wind2012.npy')
wind2012new = wind2012[:-23,:]

In [7]:
windpower = wind2012new[:,0].reshape(365,-1)*55

In [8]:
def PreProcessing(X,Xwind,H=24):

    X = X.reshape(-1,H)
    Y = X[3:,:]
    Xf = np.zeros((Y.shape[0]*H,4))
    
    for i in range(Y.shape[0]):
        for j in range(H):
            if j < 23:
                Xf[i*H+j,:] = np.array([X[i,j],X[i+1,j],X[i+1,j+1],X[i+2,j]])
            else:
                Xf[i*H+j,:] = np.array([X[i,j],X[i+1,j],X[i+2,0],X[i+2,j]])
            
#             if j < 23 & j>0:               
#                 Xf[i*H+j,:] = np.array([X[i,j],X[i+1,j],X[i+1,j+1],X[i+2,j],X[i+2,j-1]])
#             elif j == 23:
#                 Xf[i*H+j,:] = np.array([X[i,j],X[i+1,j],X[i+2,0],X[i+2,j],X[i+2,j-1]])
#             elif j == 0:
#                 Xf[i*H+j,:] = np.array([X[i,j],X[i+1,j],X[i+1,j+1],X[i+2,j],X[i+1,23]])
    
    trainsize = int(Y.shape[0]*0.8)*24
    
    Xwindnew = Xwind[3*H:,:]
    
    print(Xf.shape,Xwindnew.shape)
    
    #Xf = np.hstack((Xl,Xwind[3*H:,:]))
    
    Xmin = np.min(np.hstack((Xf,Xwindnew))[:trainsize,:],axis = 0)
    Xmax = np.max(np.hstack((Xf,Xwindnew))[:trainsize,:],axis = 0)
    
    Ymin = np.min(Y.reshape(-1)[:trainsize,])
    Ymax = np.max(Y.reshape(-1)[:trainsize,])
    
    XTrR = (np.hstack((Xf,Xwindnew))[:trainsize,:]-Xmin)/(Xmax-Xmin)
    XTeR = (np.hstack((Xf,Xwindnew))[trainsize:,:]-Xmin)/(Xmax-Xmin)
    
    YTrR = (Y.reshape(-1)[:trainsize,]-Ymin)/(Ymax-Ymin)
    YTeR = (Y.reshape(-1)[trainsize:,]-Ymin)/(Ymax-Ymin)
    
    
    
    return XTrR,XTeR,YTrR,YTeR,Y.reshape(-1)[:trainsize,],Y.reshape(-1)[trainsize:,],Ymin,Ymax


    


def DeProcessing(X,Xmin,Xmax):
    
    return (X)*(Xmax-Xmin)+Xmin

In [9]:
XTrR,XTeR,YTrR,YTeR,YTr,YTe,Ymin,Ymax = \
PreProcessing((TariffDicToUEst-windpower)*1.2,wind2012new[:,1:],H=24)

(8688, 4) (8688, 4)


In [10]:
def getscenarios(XTrR,K,X,YTr):
    neigh = NearestNeighbors(n_neighbors=K)
    neigh.fit(XTrR)

    scenarios = np.zeros((X.shape[0],K))
    for i in range(X.shape[0]):
        index = neigh.kneighbors(X[i,:][np.newaxis,:])[1].reshape(-1)
        for k in range(K):
            scenarios[i,k] = YTr[index[k]]
    return scenarios

In [11]:
scenariosnet = getscenarios(XTrR,200,XTeR,YTr)

In [12]:
print(scenariosnet.shape)

(1752, 200)


In [13]:
def twostageLP_temporal(scenarios,true,K,\
                        cd1=18,cd2=16,cu1=55,cu2=60,Beta=0.3):
    #print(scenarios.shape)
    m=grb.Model("toyDA")
    m.Params.LogToConsole = 0
    
    Pg1DA = m.addVar(vtype=GRB.CONTINUOUS)
    Pg2DA = m.addVar(vtype=GRB.CONTINUOUS)
    
    m.addConstr(Pg1DA <= 70)
    m.addConstr(-Pg1DA <= 0)
    
    m.addConstr(Pg2DA <= 70)
    m.addConstr(-Pg2DA <= 0)
    
    prediction = m.addVar(lb=0,vtype=GRB.CONTINUOUS)
    
    m.addConstr(Pg1DA + Pg2DA == prediction)
    
    Pup1_xi = m.addVars(range(K),lb = 0, ub = 60,vtype=GRB.CONTINUOUS)
    Pup2_xi = m.addVars(range(K),lb = 0, ub = 60,vtype=GRB.CONTINUOUS)
    Pdown1_xi = m.addVars(range(K),lb = 0, ub = 60,vtype=GRB.CONTINUOUS)
    Pdown2_xi = m.addVars(range(K),lb = 0, ub = 60,vtype=GRB.CONTINUOUS)
    
    
    eta_xi = m.addVars(range(K),lb = 0, vtype=GRB.CONTINUOUS)
    
    alpha = m.addVar(lb=-GRB.INFINITY,vtype=GRB.CONTINUOUS)
    s = 0
    
    for i in range(K):
        m.addConstr(-Pdown1_xi[i] - Pdown2_xi[i] + Pup1_xi[i] +\
                        Pup2_xi[i]  == scenarios[i] - prediction)
        
        m.addConstr(-cd1*Pdown1_xi[i] - cd2*Pdown2_xi[i] + cu1*Pup1_xi[i] + \
                    cu2*Pup2_xi[i] - alpha <= eta_xi[i])
        s = s + eta_xi[i]
    
    obj = 25*Pg1DA + 30*Pg2DA + alpha + s/((1-Beta)*K)
    
    m.setObjective(obj, GRB.MINIMIZE)
    m.update()
    m.optimize()
    
    Pg1DA = Pg1DA.x
    Pg2DA = Pg2DA.x
    prediction = prediction.x
    
    m1=grb.Model("toyRT")
    m1.Params.LogToConsole = 0
    
    Pup1 = m1.addVar(lb = -GRB.INFINITY, vtype=GRB.CONTINUOUS)
    Pup2 = m1.addVar(lb = -GRB.INFINITY, vtype=GRB.CONTINUOUS)
    Pdown1 = m1.addVar(lb = -GRB.INFINITY, vtype=GRB.CONTINUOUS)
    Pdown2 = m1.addVar(lb = -GRB.INFINITY, vtype=GRB.CONTINUOUS)
    
  
    
    m1.addConstr(Pup1 <= 60)
    m1.addConstr(-Pup1 <= 0)
    
    m1.addConstr(Pup2 <= 60)
    m1.addConstr(-Pup2 <= 0)
    
    m1.addConstr(Pdown1 <= 60)
    m1.addConstr(-Pdown1 <= 0)
    
    m1.addConstr(Pdown2 <= 60)
    m1.addConstr(-Pdown2 <= 0)
        

    
    m1.addConstr(-Pdown1 - Pdown2 + Pup1 + Pup2 + prediction - true == 0)
    
    
    obj = -cd1*Pdown1 - cd2*Pdown2 + cu1*Pup1 + cu2*Pup2
    
    m1.setObjective(obj, GRB.MINIMIZE)
    m1.update()
    m1.optimize()
    
    return 25*Pg1DA + 30*Pg2DA + m1.objVal,25*Pg1DA + 30*Pg2DA,m1.objVal
    

In [14]:
def cal_cvar(cost,BETA):
    #cost = cost.reshape(-1)
    cost = np.array(cost)
    quantile = np.quantile(cost,BETA)
    print(quantile)
    #print(quantile)
    largecost = []
    for i in range(cost.shape[0]):
        if cost[i]> quantile:
            largecost.append(cost[i])
            
    print(np.mean(largecost))

Beta = 0.3

In [15]:
%%time
score = []
scoreDA = []
scoreRT = []

for i in range(YTe.shape[0]):   
    overallcost,costda,costrt = twostageLP_temporal(scenariosnet[i],true = YTe[i],K=200,\
                                                    Beta=0.3)
    score.append(overallcost)
    scoreDA.append(costda)
    scoreRT.append(costrt)
print(np.mean(score))
print(np.mean(scoreDA))
print(np.mean(scoreRT))

Set parameter Username
Academic license - for non-commercial use only - expires 2024-07-09
1746.1919339320095
1909.5601583034138
-163.36822437140444
Wall time: 1min 4s


In [16]:
cal_cvar(score,BETA=0.3)

1536.2928601885553
1976.3572289142146


Beta = 0.5

In [15]:
%%time
score = []
scoreDA = []
scoreRT = []

for i in range(YTe.shape[0]):   
    overallcost,costda,costrt = twostageLP_temporal(scenariosnet[i],true = YTe[i],K=200,\
                                                    Beta=0.5)
    score.append(overallcost)
    scoreDA.append(costda)
    scoreRT.append(costrt)
print(np.mean(score))
print(np.mean(scoreDA))
print(np.mean(scoreRT))

Set parameter Username
Academic license - for non-commercial use only - expires 2024-07-09
1754.101526839765
1969.885808306509
-215.78428146674423
Wall time: 59.6 s


In [16]:
cal_cvar(score,BETA=0.5)

1766.6319017512335
2101.80047334836


Beta = 0.7

In [15]:
%%time
score = []
scoreDA = []
scoreRT = []

for i in range(YTe.shape[0]):   
    overallcost,costda,costrt = twostageLP_temporal(scenariosnet[i],true = YTe[i],K=200,\
                                                    Beta=0.7)
    score.append(overallcost)
    scoreDA.append(costda)
    scoreRT.append(costrt)
print(np.mean(score))
print(np.mean(scoreDA))
print(np.mean(scoreRT))

Set parameter Username
Academic license - for non-commercial use only - expires 2024-07-09
1770.4068358423485
2054.789749783918
-284.38291394157005
Wall time: 1min


In [17]:
cal_cvar(score,BETA=0.7)

1983.6030824452869
2239.179045476978


Beta = 0

In [53]:
%%time
score = []
scoreDA = []
scoreRT = []

for i in range(YTe.shape[0]):   
    overallcost,costda,costrt = twostageLP_temporal(scenariosnet[i],true = YTe[i],K=200,\
                                                    Beta=0)
    score.append(overallcost)
    scoreDA.append(costda)
    scoreRT.append(costrt)
print(np.mean(score))
print(np.mean(scoreDA))
print(np.mean(scoreRT))

1572.8832174407669
1828.114301399907
-255.23108395914005
Wall time: 58.3 s


In [54]:
cal_cvar(score,BETA=0)

221.04002757234161
1573.6552580974596


In [18]:
np.savetxt('./resultsNetVPP/Beta7/score_sto.txt',score)
np.savetxt('./resultsNetVPP/Beta7/scoreDA_sto.txt',scoreDA)
np.savetxt('./resultsNetVPP/Beta7/scoreRT_sto.txt',scoreRT)